# 1.  Data Preprocessing

#### Importing Libraries

This section imports necessary libraries for video processing (cv2), numerical operations (numpy), and loading the pre-trained model (tensorflow.keras.models).

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#### Loading the Pre-trained Model

Here, we load a pre-trained emotion detection model. The model is stored in the file emotion_detection_model_96.h5.

In [8]:
model = load_model('emotion_detection_model_96.h5')

#### Loading the Haar Cascade Classifier for face detection

In [9]:
# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#### Preprocessing Function

This function preprocesses each video frame by resizing, converting to grayscale, converting back to RGB, normalizing, and expanding dimensions to fit the model's input requirements.

In [10]:
def preprocess_frame(frame):
    # Resize frame to match model's expected sizing (48x48)
    frame = cv2.resize(frame, (48, 48))
    # Convert to grayscale (if needed)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Convert grayscale to RGB
    frame_rgb = cv2.cvtColor(frame_gray, cv2.COLOR_GRAY2RGB)
    # Normalize
    frame_rgb = frame_rgb / 255.0
    # Expand dimensions to match model's input shape
    frame_rgb = np.expand_dims(frame_rgb, axis=0)
    return frame_rgb

# 2. Data Visualization


#### Processing Video File

This section reads the video file frame-by-frame, detects faces using the Haar Cascade classifier, preprocesses the face regions, performs emotion predictions using the pre-trained model, and displays the results with bounding boxes and emotion labels.

In [ ]:
video_path = 'emotion_data.mp4'  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Loop through frames
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale for face detection
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(frame_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_roi = frame[y:y+h, x:x+w]
        
        # Preprocess the face region for emotion prediction
        processed_face = preprocess_frame(face_roi)
        
        # Perform emotion prediction
        prediction = model.predict(processed_face)
        predicted_class = np.argmax(prediction)
        emotion_label = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'][predicted_class]
        
        # Overlay bounding box and emotion label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f'{emotion_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Display the frame with bounding boxes and emotion labels
    cv2.imshow('Emotion Detection', frame)
    
    # Introduce a delay (1 millisecond here, adjust as needed)
    # Press 'q' to exit or adjust the delay for slower playback
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv2.destroyAllWindows()

# 3. Data Implementation


#### Processing Webcam Video

This section captures video from the webcam, processes each frame to detect faces, performs emotion predictions, and displays the results. The main difference from the previous section is the source of video input (webcam vs. video file).

In [ ]:
cap = cv2.VideoCapture(0)  # Use 0 for webcam, or replace with your video device index

if not cap.isOpened():
    print("Error: Could not access webcam or video device.")
    exit()

# Loop to capture frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame.")
        break
    
    # Convert frame to grayscale for face detection
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(frame_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face_roi = frame[y:y+h, x:x+w]
        
        # Preprocess the face region for emotion prediction
        processed_face = preprocess_frame(face_roi)
        
        # Perform emotion prediction
        prediction = model.predict(processed_face)
        predicted_class = np.argmax(prediction)
        emotion_label = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'][predicted_class]
        
        # Overlay bounding box and emotion label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, f'{emotion_label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Display the frame with bounding boxes and emotion labels
    cv2.imshow('Emotion Detection', frame)
    
    # Check for 'q' key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam or capture device and close all windows
cap.release()
cv2.destroyAllWindows()

# Conclusion

This notebook demonstrates how to preprocess video frames, detect faces, predict emotions using a pre-trained model, and visualize the results in real-time. It includes code for processing both video files and live webcam feeds.